In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
trips = pd.read_csv('../../CSVs/trips.csv', low_memory=False)
trip_updates = pd.read_csv('../../CSVs/trip_updates.csv', low_memory=False)
vehicle_pos = pd.read_csv('../../CSVs/vehicle_positions.csv', low_memory=False)
stop_times = pd.read_csv('../../CSVs/stop_times.csv', dtype={'trip_id': str, 'stop_id': str})
stop_times.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00,NaN,0,1,NaN,1.0
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,08:04:00,08:04:00,NaN,0,0,NaN,1.0
2,Logan-22-Weekday-trip,Logan-A,3,08:09:00,08:09:00,NaN,0,0,NaN,1.0
3,Logan-22-Weekday-trip,Logan-B,4,08:12:00,08:12:00,NaN,0,0,NaN,1.0
4,Logan-22-Weekday-trip,Logan-Subway,5,08:17:00,08:17:00,NaN,1,0,NaN,1.0


In [5]:
trips.head()

,trip_id,route_id,service_id,direction_id,block_id,shape_id,trip_type,trip_headsign,trip_short_name,bikes_allowed,wheelchair_accessible
0,Logan-22-Weekday-trip,Logan-22,Logan-Weekday,0,NaN,NaN,NaN,Loop,NaN,0,1
1,Logan-22-Weekend-trip,Logan-22,Logan-Weekend,0,NaN,NaN,NaN,Loop,NaN,0,1
2,Logan-33-Weekday-trip,Logan-33,Logan-Weekday,0,NaN,NaN,NaN,Loop,NaN,0,1
3,Logan-33-Weekend-trip,Logan-33,Logan-Weekend,0,NaN,NaN,NaN,Loop,NaN,0,1
4,Logan-55-Weekday-trip,Logan-55,Logan-Weekday,0,NaN,NaN,NaN,Loop,NaN,0,1


In [6]:
trips = trips.loc[:, ['route_id', 'trip_id', 'trip_headsign']]
trips.head()

,route_id,trip_id,trip_headsign
0,Logan-22,Logan-22-Weekday-trip,Loop
1,Logan-22,Logan-22-Weekend-trip,Loop
2,Logan-33,Logan-33-Weekday-trip,Loop
3,Logan-33,Logan-33-Weekend-trip,Loop
4,Logan-55,Logan-55-Weekday-trip,Loop


In [7]:
route_stops = stop_times.merge(trips, on='trip_id', how='inner')
route_stops.head()
#route_stops['route_id'].unique()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,route_id,trip_headsign
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00,NaN,0,1,NaN,1.0,Logan-22,Loop
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,08:04:00,08:04:00,NaN,0,0,NaN,1.0,Logan-22,Loop
2,Logan-22-Weekday-trip,Logan-A,3,08:09:00,08:09:00,NaN,0,0,NaN,1.0,Logan-22,Loop
3,Logan-22-Weekday-trip,Logan-B,4,08:12:00,08:12:00,NaN,0,0,NaN,1.0,Logan-22,Loop
4,Logan-22-Weekday-trip,Logan-Subway,5,08:17:00,08:17:00,NaN,1,0,NaN,1.0,Logan-22,Loop


These are all the routes that trips are associated with in the stop_times table

In [8]:
route_stops['route_id'].value_counts()[-15:]

351         360
8993        333
434         264
3738        231
Boat-F4     224
171         160
3233         64
9701         30
Logan-55     22
9702         22
Logan-66     22
9703         20
195          15
Logan-33     14
Logan-22     14
Name: route_id, dtype: int64

We can see how many trips there are per route

In [9]:
route_stops = route_stops.loc[:, ['trip_id', 'stop_id', 'stop_sequence', 'route_id']]
route_stops.head()

,trip_id,stop_id,stop_sequence,route_id
0,Logan-22-Weekday-trip,Logan-Subway,1,Logan-22
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,Logan-22
2,Logan-22-Weekday-trip,Logan-A,3,Logan-22
3,Logan-22-Weekday-trip,Logan-B,4,Logan-22
4,Logan-22-Weekday-trip,Logan-Subway,5,Logan-22


In [10]:
total_trips = route_stops['trip_id'].value_counts().index
print('There are {} trips'.format(len(total_trips)))

There are 72342 trips


In [11]:
total_trips

Index(['35065459', '35065445', '35065461', '35065451', '35065455', '35065453',
       '35065465', '35271201', '35065443', '35065457',
       ...
       'Boat-F1-1800-Rowes-Weekday', 'CR-Weekday-Spring-17-289',
       'Boat-F1-0650-Hingham-Weekday', 'Boat-F4-0900-Charlestown-Weekday',
       'Boat-F1-0630-Hingham-Weekday', 'Boat-F1-0805-Hull-SummerWeekday',
       'Boat-F1-1740-Rowes-SummerWeekday', 'Boat-F1-1930-Rowes-Weekday',
       'Boat-F1-0630-Hingham-SummerWeekday', 'Boat-F4-1230-Long-Weekend'],
      dtype='object', length=72342)

## Robust Way of Obtaining Patterns

The following cells display code that is used to obtain the unique trip patterns that exist under a particular target route. It is dubbed 'Robust', since it involves a thorough method that is sure to obtain all the patterns and label each trip with the appropiate patter id. The steps are as follows
<ol>
    <li>obtain a list of trips within a target route</li>
    <li>for every trip obtain a python tuple of stops</li>
    <li>add this tuple to a grand set of stops </li>
    <li>use this tuple as a key in a dictionary (only if it is not already)</li>
</ol>
Note that by using a set we take care of the case where different iterations can have the same tuple of stops.

In [12]:
# lets focus on just a subset of route_stops
# in particular lets focus on a route instead
# of all routes
def get_targ_trips(targ_route, route_stops):
    return route_stops[route_stops['route_id'] == targ_route].reset_index(drop=True)
trg_trps = get_targ_trips('448', route_stops)
trg_trps.head()

,trip_id,stop_id,stop_sequence,route_id
0,35058168,4807,1,448
1,35058168,4832,2,448
2,35058168,4834,3,448
3,35058168,14836,6,448
4,35058168,4835,4,448


In [13]:
# display all the trips that are under this route
route_trips = trg_trps['trip_id'].unique()
route_trips

array(['35058168', '35058169', '35058204', '35058279', '35058280',
       '35070565', '35070566', '35070601', '35070676', '35070677'], dtype=object)

In [15]:


# get_trip_pats will return a mapping 
# of the form stops: pattern_name where
# stops is a tuple that consists of the 
# stop_ids a trip travels through and
# pattern_name is the name of the pattern
def get_trip_pts(route_trips, targ_trips, route, debug = False):
    trip_patterns_dict = {}
    alph_iter = iter(range(ord('A'), ord('Z')))
    alph_iter2 = iter(range(ord('A'), ord('Z')))
    trip_patterns = set()
    cur_let = chr(next(alph_iter))
    
    for tr in route_trips:
        stops = targ_trips[targ_trips['trip_id'] == tr].sort_values('stop_sequence')['stop_id']
        stops = tuple(stops)
        if debug:
            print('{}\n'.format(tr))
            print(tuple(stops))
        trip_patterns.add(stops)
        if stops not in trip_patterns_dict:
            try:
                trip_patterns_dict[stops] = '{}_{}{}'.format(route, cur_let, chr(next(alph_iter2)))
            except StopIteration:
                alph_iter2 = iter(range(ord('A'), ord('Z')))
                cur_let = chr(next(alph_iter))
                trip_patterns_dict[stops] = '{}_{}{}'.format(route, cur_let, chr(next(alph_iter2)))
                
    return trip_patterns, trip_patterns_dict

trp_pat, trp_pat_dict = get_trip_pts(route_trips, trg_trps, '448')
#trip_patterns_dict = {name: trp for trp, name in trip_patterns_dict.items()}
#trip_patterns_dict['448A'][:10]
trp_pat_dict.values()

dict_values(['448_AA', '448_AB', '448_AC'])

In [16]:
# make a new column for pattern id
trg_trps['pattern_id'] = ''

# fill_trp_pat will fill in the pattern_id column
# of all the rows in targ_trips with a matching trip_id
def fill_trp_pat(route_trips, targ_trips, trip_patterns_dict):
    # iterate through every trip
    for tr in route_trips:
        # find the rows that have a matching trip_id
        match_trps = targ_trips[targ_trips['trip_id'] == tr]
        # get the tuple that will contain the stops for this trip
        stops = tuple(match_trps.sort_values('stop_sequence')['stop_id'])
        # use the dictionary to get the pattern name based on the stops
        pat_name = trip_patterns_dict[stops]    
        # fill in all the rows with the appropiate pattern name
        targ_trips.loc[targ_trips['trip_id'] == tr, 'pattern_id'] = pat_name
        
fill_trp_pat(route_trips, trg_trps, trp_pat_dict)
trg_trps['pattern_id'].unique()

array(['448_AA', '448_AB', '448_AC'], dtype=object)

In [17]:
gp = trg_trps.groupby('pattern_id')
gp['trip_id'].unique()

pattern_id
448_AA    [35058168, 35058169, 35070565, 35070566]
448_AB                        [35058204, 35070601]
448_AC    [35058279, 35058280, 35070676, 35070677]
Name: trip_id, dtype: object

Here we can see that we only used the pattern names that we generated as well as the trip_ids that belong to each pattern.

In [18]:
rt_stops = route_stops.loc[:, ['route_id', 'stop_id', 'stop_sequence', 'trip_id']]
rt_stops.head()

,route_id,stop_id,stop_sequence,trip_id
0,Logan-22,Logan-Subway,1,Logan-22-Weekday-trip
1,Logan-22,Logan-RentalCarCenter,2,Logan-22-Weekday-trip
2,Logan-22,Logan-A,3,Logan-22-Weekday-trip
3,Logan-22,Logan-B,4,Logan-22-Weekday-trip
4,Logan-22,Logan-Subway,5,Logan-22-Weekday-trip


In [19]:
targ_routes = trips['route_id'].unique()
len(targ_routes)

212

In [61]:
ptn_trips = []

for route in targ_routes:
    # obtain all records associated with this route in the stop_times table
    targ_trps = get_targ_trips(route, rt_stops)
    # grab all the trips from this collection of records
    route_trips = targ_trps['trip_id'].unique()
    # obtain the set of unique trip patterns as well as a dictionary
    # with a mapping of the trip pattern to its name
    pat_set, pat_dict = get_trip_pts(route_trips, targ_trps, route)
    #  fill in the pattern_id column of all the rows in targ_trips with a matching trip_id
    fill_trp_pat(route_trips, targ_trps, pat_dict)
    # object that groups trips in this route by their pattern_id
    gp = targ_trps.groupby('pattern_id')['trip_id'].unique()
    # save this grouping to the result list
    ptn_trips.append(gp)

In [76]:
ptn_trips[0]

pattern_id
Logan-22_AA    [Logan-22-Weekday-trip, Logan-22-Weekend-trip]
Name: trip_id, dtype: object

In [81]:
trips_wpats = pd.DataFrame(columns=['trip_id', 'pattern_id'])
for i in ptn_trips:
    
    temp_df = None
    for pat_id, trps in i.items():
        df_dict = {'pattern_id': [pat_id for i in range(0, len(trps))]}
        df_dict['trip_id'] = trps
        temp_df = pd.DataFrame.from_dict(df_dict)
        trips_wpats = pd.concat([trips_wpats, temp_df], ignore_index=True)



In [82]:
trips_wpats.head()

,pattern_id,trip_id
0,Logan-22_AA,Logan-22-Weekday-trip
1,Logan-22_AA,Logan-22-Weekend-trip
2,Logan-33_AA,Logan-33-Weekday-trip
3,Logan-33_AA,Logan-33-Weekend-trip
4,Logan-55_AA,Logan-55-Weekday-trip


In [83]:
trips_wpats.shape

(72342, 2)

In [84]:
trips_wpats.to_csv('trips_with_pats.csv', index=False)

## Obtain Patterns with Headsign

The following cells attempt to reach the same results as the Robust method. Unfortunately it looks like we cannot rely on the headsign to give us an accurate distinction between different patterns.

In [126]:
pat_448A = gp['trip_id'].unique()['448A']
trips[(trips['trip_id'] == '35058168') |  (trips['trip_id'] == '35058169') | (trips['trip_id'] == '35070565') | (trips['trip_id'] == '35070566')]

,route_id,trip_id,trip_headsign
28492,448,35058168,Downtown Express via Airport and Paradise Rd
28493,448,35058169,Downtown Express via Airport and Paradise Rd
30704,448,35070565,Downtown Express via Airport and Paradise Rd
30705,448,35070566,Downtown Express via Airport and Paradise Rd


It looks like all trips from 448A have the same headsign, which is good!

In [128]:
trips[(trips['trip_id'] == '35058204') | (trips['trip_id'] == '35070601')]

,route_id,trip_id,trip_headsign
28528,448,35058204,Marblehead via Paradise Rd & Central Sq (Express)
30740,448,35070601,Marblehead via Paradise Rd & Central Sq (Express)


All the trips in 448B also share the same headsign and differ from the headsign seen in 448A

In [140]:
trips[(trips['trip_id'] =='35058279') | (trips['trip_id'] =='35058280') | (trips['trip_id'] == '35070676') | (trips['trip_id'] == '35070677')]

,route_id,trip_id,trip_headsign
28603,448,35058279,Marblehead via Paradise Rd & Central Sq (Express)
28604,448,35058280,Marblehead via Paradise Rd & Central Sq (Express)
30815,448,35070676,Marblehead via Paradise Rd & Central Sq (Express)
30816,448,35070677,Marblehead via Paradise Rd & Central Sq (Express)


It looks like all the trips from 448C share the same headsign as those from 448B. Perhaps the Robust method does not yield the correct list of patterns?

In [134]:
pat_name_dict = {name:stops for stops, name in trp_pat_dict.items()}
set(pat_name_dict['448B']) - set(pat_name_dict['448C'])

{'24717',
 '4709',
 '4717',
 '4727',
 '4728',
 '4730',
 '4731',
 '4732',
 '4735',
 '4736',
 '4738',
 '4739',
 '4741',
 '4742',
 '4743',
 '4744',
 '4745',
 '4746',
 '4747',
 '4854',
 '48541',
 '4855',
 '4856'}

In [136]:
'4709' in set(pat_name_dict['448C'])

False

It turns out that there is a difference between the stops that 448B and 448C visit! Therefore we cannot rely on the headsign as an alternative to determining patterns